In [ ]:
import pandas as pd
import re

from utils.column_merge_utils import merge_cols_and_place, column_merge
from utils.table_merge_utils import parse_legal_addr, parse_street_addr

In [ ]:
# 주소 파싱 컬럼명 리스트
legal_addr_cols = ['법정동주소(시도)','법정동주소(군구)','법정동주소(동리)','법정동주소(번지)', '법정동주소(상세)', '법정동주소(중복주소)']
street_addr_cols = ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)', '도로명주소(중복주소)']

# 병합키로 쓰이는 컬럼명 리스트
legal_addr_merge_keys = ['법정동주소(시도)','법정동주소(군구)','법정동주소(동리)','법정동주소(번지)']
street_addr_merge_keys = ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)']

In [ ]:
column_merge_rules = [
    [['단지명', '건물명'], ['단지명']],
    [['세대수', '세대수(세대)'], ['세대수', '보조_세대수']],
    [['건물구조', '구조코드명'], ['건물구조', '보조_건물구조']],
    [['최고층수(건축물대장상)', '지상층수'], ['최고층수(건축물대장상)']],
    [['k-연면적', '연면적(㎡)'], ['연면적', '보조_연면적']],
    [['건축면적', '건축면적(㎡)'], ['건축면적', '보조_건축면적']],
]

# 파일 읽기 및 전처리
파일 읽기 및 디버깅을 위한 아이디값 부여<br>
kapt와 sapt(서울시공공주택정보)를 join한 테이블은 apt라고 이름붙였음

In [ ]:
apt_excel_path = '../input/kapt_sapt_병합.xlsx'
apt_csv_path = '../input/kapt_sapt_병합.csv'

blds_excel_path = '../input/건축물대장_주용도필터_디자인추가.xlsx'
blds_csv_path = '../input/blds.csv'

In [ ]:
# apt = pd.read_excel(apt_excel_path)
# apt.to_csv(apt_csv_path, index=False)

# blds = pd.read_excel(blds_excel_path)
# blds.to_csv(blds_csv_path, index=False)

In [ ]:
apt = pd.read_csv(apt_csv_path)
print(f"apt: {len(apt)}")

blds = pd.read_csv(blds_csv_path)
print(f"blds: {len(blds)}")

# id 생성
apt.insert(0, 'apt_id', range(1, len(apt) + 1))
blds.insert(0, 'blds_id', range(1, len(blds) + 1))

# ['병합종류'] 컬럼 수정
apt = apt.rename(columns={'병합종류':'apt_병합종류'})
apt['apt_병합종류'] = apt['apt_병합종류'].replace('성공', '2번,3번')
blds.insert(0, 'blds_병합종류', '1번')

# 주소 파싱

In [ ]:
apt[
    legal_addr_cols
] = apt['법정동주소'].apply(
    lambda x: pd.Series(parse_legal_addr(x))
)   

blds[
    legal_addr_cols
] = blds['대지위치'].apply(
    lambda addr: pd.Series(parse_legal_addr(addr))
)   

apt[
    street_addr_cols
] = apt['도로명주소'].apply(
    lambda x: pd.Series(parse_street_addr(x))
)

blds[
    street_addr_cols
] = blds['도로명대지위치'].apply(
    lambda x: pd.Series(parse_street_addr(x))
)

# 병합

In [ ]:
# apt, blds merge
def merge_apt_with_blds(
    apt_df,     # apt
    blds_df,    # blds
    on_list     # 매칭의 기준이 되는 컬럼 리스트
    ):
    suffix = '_suffix'
    
    # LEFT JOIN
    merged = apt_df.merge(
        blds_df.dropna(subset=on_list, how='all'),
        on = on_list,
        how = 'left',
        suffixes = ('', suffix),  # 컬럼명이 같은 경우 출처가 blds인 컬럼명 뒤에 suffix 붙임
        indicator = True
    )
    
    # suffix가 붙은 컬럼 병합
    cols_to_column_merge = {c.removesuffix(suffix):c for c  in merged.columns if c.endswith(suffix)}
    for col1, col2 in cols_to_column_merge.items():
        merged = merge_cols_and_place(merged, [col1,col2], [col1])
    
    matched = ( # LEFT JOIN 결과물 중 양쪽 매칭에 성공한 것
    merged.loc[merged['_merge'] == 'both']
    .drop(columns=['_merge'])
    .copy()
    )
    
    unmatched_apt = ( # 매칭에 실패한 apt 데이터
        merged.loc[merged['_merge'] == 'left_only']
        .drop(columns=['_merge'])
        .copy()
    )
    unmatched_apt = unmatched_apt.drop( # unmatched_apt를 다음 매칭에 쓰기 위해 출처가 blds인 컬럼 제거
        columns=[c for c in unmatched_apt.columns if c not in apt_df.columns]
    )

    return matched, unmatched_apt

In [ ]:
# 우선적으로는 총괄표제부만 매칭에 참여
총괄_blds = blds[blds['그룹 종류'] == 0]
print(f"총괄_blds : {len(총괄_blds)}")
print(f"apt : {len(apt)}")


# 법정동주소로 매칭
matched_on_법정동주소, unmatched_apt = merge_apt_with_blds(apt, 총괄_blds, legal_addr_merge_keys)
matched_on_법정동주소['병합종류'] = '총괄,법정동'
print(f"matched_on_법정동주소 : {len(matched_on_법정동주소)}")


# 도로명주소로 매칭
matched_on_도로명주소, unmatched_apt = merge_apt_with_blds(unmatched_apt, 총괄_blds, street_addr_merge_keys)
matched_on_도로명주소['병합종류']='총괄,도로명'
print(f"matched_on_도로명주소 : {len(matched_on_도로명주소)}")


matched = pd.concat([matched_on_도로명주소,matched_on_법정동주소], ignore_index=True)                   # 매칭 성공한 것
merged = pd.concat([matched, unmatched_apt], ignore_index=True)    # 매칭 성공한 것 + 매칭 실패한 apt
print(f"unmatched : {len(unmatched_apt)}")
print(f"matched : {len(matched)}")
print(f"merged : {len(merged)}")

In [ ]:
# 총괄표제부를 가지지 않는 일반표제부만 매칭에 참여
일반_blds = blds[blds['그룹 종류'] == 2] 


# 법정동주소로 매칭
matched_on_법정동주소, unmatched_apt = merge_apt_with_blds(unmatched_apt, 일반_blds, legal_addr_merge_keys)
matched_on_법정동주소['병합종류'] = '일반,법정동'
print(f"matched_on_법정동주소 : {len(matched_on_법정동주소)}")


# 도로명주소로 매칭
matched_on_도로명주소, unmatched_apt = merge_apt_with_blds(unmatched_apt, 일반_blds, street_addr_merge_keys)
matched_on_도로명주소['병합종류']='일반,도로명'
print(f"matched_on_도로명주소 : {len(matched_on_도로명주소)}")


matched = pd.concat(
    [matched, matched_on_법정동주소, matched_on_도로명주소],
    ignore_index=True
)
merged = pd.concat(
    [matched, unmatched_apt],
    ignore_index=True
)
print(f"unmatched : {len(unmatched_apt)}")
print(f"matched: {len(matched)}")
print(f"merged : {len(merged)}")

In [ ]:
# FULL_JOIN : 매칭에 실패한 apt, blds 데이터 전부 포함
full_merged = pd.concat(
    [merged, blds]
)

full_merged = full_merged.drop_duplicates().reset_index(drop=True)  
print(len(full_merged))

# 컬럼 병합

In [ ]:
full_merged = column_merge(full_merged, column_merge_rules)

#### 도로명주소 컬럼 병합

In [ ]:
def norm_도로명주소(addr : str):                    # 도로명주소 컬럼 정규화
    if not isinstance(addr, str):
        return addr
    addr = re.sub(r'\s*\([^)]*\)', '', addr)     # 괄호와 그 안의 값 제거
    addr = addr.strip()                          # 양쪽의 공백 제거
    return addr
for col in ['도로명주소', '보조_도로명주소', '도로명대지위치']:   
    full_merged[col] = full_merged[col].apply(norm_도로명주소)

full_merged = column_merge(full_merged, [[['도로명주소', '보조_도로명주소', '도로명대지위치'], ['도로명주소', '보조_도로명주소']]])

def merge_도로명주소(addr:str, sub_addr:str):       # '도로명주소'가 '보조_도로명주소'를 포함하는 경우, '보조_도로명주소'는 공백으로
    if (sub_addr!='') and (sub_addr in addr):
        sub_addr = ''
    return addr, sub_addr
full_merged[['도로명주소','보조_도로명주소']] = full_merged.apply(
    lambda row: merge_도로명주소(row['도로명주소'], row['보조_도로명주소']),
    axis=1,
    result_type='expand'
)

# 테이블 정리

In [ ]:
# 불필요한 컬럼 제거
unnamed_columns = [col for col in full_merged.columns if "Unnamed" in col]
full_merged = full_merged.drop(columns=['대지위치','번', '지', '시도', '시군구', '읍면', '동리', '나머지주소', '주소(도로명)', '주소(도로상세주소)', 'apt_병합종류', 'blds_병합종류'] + unnamed_columns)

In [ ]:
# 컬럼 위치 조정
front_cols = ['병합종류', 'apt_id', 'kapt_id', 'sapt_id', 'blds_id'] + legal_addr_cols + street_addr_cols
other_cols = [c for c in full_merged.columns if c not in front_cols]

full_merged = full_merged[front_cols+other_cols]

# 디자인 추가

In [ ]:
def highlight_merge_fail(row, main_col, sub_cols, color="#ffd900"):
    styles = [''] * len(row)

    has_fail = any(
        pd.notna(row[c]) and row[c] != ''
        for c in sub_cols
        if c in row.index
    )

    if not has_fail:
        return styles

    target_cols = [main_col] + list(sub_cols)

    for i, col in enumerate(row.index):
        if col in target_cols:
            styles[i] = f'background-color: {color}'

    return styles

In [ ]:
def highlight_주소_merge_fail(row, main_col, sub_cols, color="#ffd900"):
    styles = [''] * len(row)

    has_fail = any(
        pd.notna(row[c]) and row[c] != ''
        for c in sub_cols
        if c in row.index
    )

    if not has_fail:
        return styles

    target_cols = [main_col] + list(sub_cols)

    for i, col in enumerate(row.index):
        if col in target_cols:
            styles[i] = f'background-color: {color}'

    return styles

In [ ]:
styled = full_merged.copy().style

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='단지분류',
    sub_cols=['보조_단지분류']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='도로명주소',
    sub_cols=['보조_도로명주소']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='세대수',
    sub_cols=['보조_세대수1', '보조_세대수2']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='연면적',
    sub_cols=['보조_연면적']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='건축면적',
    sub_cols=['보조_건축면적']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='관리방식',
    sub_cols=['보조_관리방식']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='난방방식',
    sub_cols=['보조_난방방식']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='경비관리방식',
    sub_cols=['보조_경비관리방식']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='청소관리방식',
    sub_cols=['보조_청소관리방식']
)

# 결과

In [ ]:
full_merged.to_csv("../output/apt_blds_병합.csv")
print("csv created")
full_merged.to_excel("../output/apt_blds_병합.xlsx", index=False)
print("excel created")
# styled.to_excel("../output/apt_blds_병합_디자인추가.xlsx", index=False)
# print("designed excel created")
# unmatched_apt.to_excel("../output/apt_병합실패.xlsx", index=False)